## PHÂN TÍCH DỮ LIỆU MỨC TIÊU THỤ NHIÊN LIỆU

## Importing libraries
https://www.kaggle.com/datasets/uciml/autompg-dataset?select=auto-mpg.csv

In [30]:
import pandas as pd
import tensorflow as tf; layers = tf.keras.layers
print(tf.__version__)

2.18.0


### Đọc dataset auto-mpg.csv

In [32]:
from pandas.core import indexing
column_names = ['MPG','Cylinders','Displacement','Horsepower','Weight',
                'Acceleration', 'Model Year', 'Origin']

raw_dataset = pd.read_csv('auto-mpg.csv', names=column_names,
                      na_values = "?", comment='\t',
                      sep=",",index_col=False,skiprows=1)

dataset = raw_dataset.copy()
dataset.tail()

/var/folders/nr/t52zs2xx4qlg1wz53sp17qyh0000gn/T/ipykernel_20668/2175335280.py:5: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  raw_dataset = pd.read_csv('auto-mpg.csv', names=column_names,


,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,Origin
393,27.0,4,140.0,86.0,2790,15.6,82,1
394,44.0,4,97.0,52.0,2130,24.6,82,2
395,32.0,4,135.0,84.0,2295,11.6,82,1
396,28.0,4,120.0,79.0,2625,18.6,82,1
397,31.0,4,119.0,82.0,2720,19.4,82,1


### Kiểm tra số lượng giá trị bị thiếu cho từng cột.

In [33]:
dataset.isna().sum()

MPG             0
Cylinders       0
Displacement    0
Horsepower      6
Weight          0
Acceleration    0
Model Year      0
Origin          0
dtype: int64

### Loại bỏ các hàng có giá trị bị thiếu.

In [34]:
dataset = dataset.dropna()
dataset

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,Origin
0,18.0,8,307.0,130.0,3504,12.0,70,1
1,15.0,8,350.0,165.0,3693,11.5,70,1
2,18.0,8,318.0,150.0,3436,11.0,70,1
3,16.0,8,304.0,150.0,3433,12.0,70,1
4,17.0,8,302.0,140.0,3449,10.5,70,1
...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86.0,2790,15.6,82,1
394,44.0,4,97.0,52.0,2130,24.6,82,2
395,32.0,4,135.0,84.0,2295,11.6,82,1
396,28.0,4,120.0,79.0,2625,18.6,82,1


In [35]:
origin = dataset.pop('Origin')
dataset['USA'] = (origin == 1) * 1.0
dataset['Europe'] = (origin == 2) * 1.0
dataset['Japan'] = (origin == 3) * 1.0
dataset.tail()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,USA,Europe,Japan
393,27.0,4,140.0,86.0,2790,15.6,82,1.0,0.0,0.0
394,44.0,4,97.0,52.0,2130,24.6,82,0.0,1.0,0.0
395,32.0,4,135.0,84.0,2295,11.6,82,1.0,0.0,0.0
396,28.0,4,120.0,79.0,2625,18.6,82,1.0,0.0,0.0
397,31.0,4,119.0,82.0,2720,19.4,82,1.0,0.0,0.0


### Chia bộ dữ liệu thành phần huấn luyện và kiểm tra.

In [36]:
train_dataset = dataset.sample(frac=0.8,random_state=0)
test_dataset = dataset.drop(train_dataset.index)

In [37]:
train_stats = train_dataset.describe()
train_stats

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,USA,Europe,Japan
count,314.000000,314.000000,314.000000,314.000000,314.000000,314.000000,314.000000,314.000000,314.000000,314.000000
mean,23.310510,5.477707,195.318471,104.869427,2990.251592,15.559236,75.898089,0.624204,0.178344,0.197452
std,7.728652,1.699788,104.331589,38.096214,843.898596,2.789230,3.675642,0.485101,0.383413,0.398712
min,10.000000,3.000000,68.000000,46.000000,1649.000000,8.000000,70.000000,0.000000,0.000000,0.000000
25%,17.000000,4.000000,105.500000,76.250000,2256.500000,13.800000,73.000000,0.000000,0.000000,0.000000
50%,22.000000,4.000000,151.000000,94.500000,2822.500000,15.500000,76.000000,1.000000,0.000000,0.000000
75%,28.950000,8.000000,265.750000,128.000000,3608.000000,17.200000,79.000000,1.000000,0.000000,0.000000
max,46.600000,8.000000,455.000000,225.000000,5140.000000,24.800000,82.000000,1.000000,1.000000,1.000000


### Tách trường đầu ra khỏi các trường khác.

In [38]:
train_labels = train_dataset.pop('MPG')
test_labels = test_dataset.pop('MPG')

In [39]:
train_stats = train_dataset.describe()
train_stats = train_stats.transpose()
train_stats

,count,mean,std,min,25%,50%,75%,max
Cylinders,314.0,5.477707,1.699788,3.0,4.00,4.0,8.00,8.0
Displacement,314.0,195.318471,104.331589,68.0,105.50,151.0,265.75,455.0
Horsepower,314.0,104.869427,38.096214,46.0,76.25,94.5,128.00,225.0
Weight,314.0,2990.251592,843.898596,1649.0,2256.50,2822.5,3608.00,5140.0
Acceleration,314.0,15.559236,2.789230,8.0,13.80,15.5,17.20,24.8
Model Year,314.0,75.898089,3.675642,70.0,73.00,76.0,79.00,82.0
USA,314.0,0.624204,0.485101,0.0,0.00,1.0,1.00,1.0
Europe,314.0,0.178344,0.383413,0.0,0.00,0.0,0.00,1.0
Japan,314.0,0.197452,0.398712,0.0,0.00,0.0,0.00,1.0


### Model Training

In [40]:
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']
normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

def build_model():
  model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[len(train_dataset.keys())]),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
  return model

In [41]:
model = build_model()
model.summary()

/opt/homebrew/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 64)             │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,865 (19.00 KB)

 Trainable params: 4,865 (19.00 KB)

 Non-trainable params: 0 (0.00 B)

In [42]:
EPOCHS = 1000
history = model.fit(
  normed_train_data, train_labels,
  epochs=EPOCHS)

Epoch 1/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 612.2312 - mae: 23.3724 - mse: 612.2312  
Epoch 2/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 563.4611 - mae: 22.3821 - mse: 563.4611 
Epoch 3/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 500.3676 - mae: 20.9718 - mse: 500.3676 
Epoch 4/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 433.7936 - mae: 19.3582 - mse: 433.7936 
Epoch 5/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 365.8389 - mae: 17.5663 - mse: 365.8389 
Epoch 6/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 290.8497 - mae: 15.6083 - mse: 290.8497 
Epoch 7/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 219.2397 - mae: 13.2555 - mse: 219.2397
Epoch 8/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 140.3445 - mae: 10.3768 - mse: 140.3445 
Epoch 9/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 88.7426 - mae: 8.0757 - mse: 88.7426
Epoch 10/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 50.8867 - mae: 5.9692 - mse: 50.88

### Model Testing

In [25]:
loss, mae, mse = model.evaluate(normed_test_data, test_labels, verbose=0)
loss,mae,mse

(7.736292839050293, 2.158836841583252, 7.736292839050293)

In [26]:
test_predictions = model.predict(normed_test_data).flatten()
test_predictions

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step


array([15.088988, 10.095673,  9.594593, 24.748722, 20.18616 , 14.100522,
       13.657548, 13.202683, 20.14071 , 32.48896 , 21.473244, 20.026455,
       15.689088, 22.340473, 13.54828 , 14.495796, 14.33884 , 14.419393,
       18.977829, 13.367166, 14.219579, 25.628105, 20.147537, 20.282455,
       30.043892, 23.63042 , 16.556778, 24.04281 , 18.611507, 17.97413 ,
       23.684752, 25.399967, 18.858395, 22.157507, 27.640633, 16.036152,
       19.756756, 25.628239, 29.14798 , 19.010323, 30.584414, 25.922838,
       15.410136, 39.245342, 37.407875, 38.546646, 19.566513, 19.967978,
       21.143192, 23.710844, 33.03965 , 16.814857, 34.403847, 17.392431,
       18.461689, 18.47216 , 35.630753, 37.564285, 21.505085, 27.171846,
       42.804955, 35.30539 , 28.14492 , 28.224506, 32.80641 , 39.074665,
       37.38889 , 36.78347 , 39.384277, 31.471405, 19.140665, 32.31552 ,
       30.928448, 30.88956 , 40.508495, 34.7144  , 37.869606, 30.411293],
      dtype=float32)

In [27]:
test_labels

9      15.0
25     10.0
28      9.0
31     25.0
33     19.0
       ... 
369    34.0
375    36.0
382    34.0
384    32.0
396    28.0
Name: MPG, Length: 78, dtype: float64

### Model Conversion

In [29]:
kearas_file = "automobile.h5"
tf.keras.models.save_model(model,kearas_file)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tfmodel = converter.convert()
open("automobile.tflite","wb").write(tfmodel)

INFO:tensorflow:Assets written to: /var/folders/nr/t52zs2xx4qlg1wz53sp17qyh0000gn/T/tmptqf6cefj/assets


INFO:tensorflow:Assets written to: /var/folders/nr/t52zs2xx4qlg1wz53sp17qyh0000gn/T/tmptqf6cefj/assets


Saved artifact at '/var/folders/nr/t52zs2xx4qlg1wz53sp17qyh0000gn/T/tmptqf6cefj'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 9), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  6061422928: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6061424272: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6061423312: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6061423504: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6061424464: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6061422736: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1738744701.510037  227294 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1738744701.510051  227294 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-02-05 15:38:21.510151: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /var/folders/nr/t52zs2xx4qlg1wz53sp17qyh0000gn/T/tmptqf6cefj
2025-02-05 15:38:21.510419: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-02-05 15:38:21.510424: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /var/folders/nr/t52zs2xx4qlg1wz53sp17qyh0000gn/T/tmptqf6cefj
2025-02-05 15:38:21.512883: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-02-05 15:38:21.527082: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /var/folders/nr/t52zs2xx4qlg1wz53sp17qyh0000gn/T/tmptqf6cefj
2025-02-05 15:38:21.531632: I tensorflow/cc/saved_model/loader.cc:

21344